In [29]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
import sys
import os
if '../src' not in sys.path:
    sys.path.append('../src')
import utils
import settings
from retriever import Retriever
from chatbot import Chatbot
from IPython.display import Markdown, display
from langchain_openai import ChatOpenAI

In [31]:
logger = utils.get_logger("03_rag")
wrapper = utils.get_text_wrapper()

### Document retriever

The document retriever is responsible for finding relevant documents based on the user question. It initializes an Opensearch client, uses a `neural` query for semantic search.

Use the `MODEL_ID` of the embedding model used at ingestion time to generate vector embeddings for the queries' text.

[OpenSearch Python Client docs](https://opensearch-project.github.io/opensearch-py/api-ref/clients/opensearch_client.html)

In [32]:
# INDEX_NAME = 'rag-knn-index'
INDEX_NAME = 'unstructured-knn-index'
retriever = Retriever(logger=logger, index=INDEX_NAME)

In [33]:
query_text = "Ce documente sunt necesare pentru pentru inscrierea la un program de studii in limba engleza?"

docs = retriever.retrieve_documents(query_text=query_text)

retriever.print_documents(docs)

{
    "page_content":"Despre testul de limbă on-line POLITEHNICA București din data de 21 martie 2025 L Ce este cu acest test? Am nevoie de el? Dacă doresc să
beneficiez de avantajele studiului într-una din limbile de largă circulație (engleză, franceză, germană) și în lista mea de opțiuni includ domenii la care
școlarizarea se desfășoară într-una din acest limbi, trebuie, conform legii, să fac dovada că am cunoștințele lingvistice necesare. Dovada se face folosind
oricare dintre variantele de mai jos:",
    "metadata":{
        "score":0.07878805,
        "filename":"Test limba straina 21 03 detalii S1 2025.pdf",
"page_number":1,
        "id":"9cf23be3e6b1bcbd56f0cab3918a505c41688bb641a64cac3a8509bc9628e5d1-1",
        "text":"Despre testul de limbă on-line POLITEHNICA
București din data de 21 martie 2025 L Ce este cu acest test? Am nevoie de el? Dacă doresc să beneficiez de avantajele studiului într-una din limbile de largă
circulație (engleză, franceză, germană) și în lista mea de o

In [34]:
display(Markdown(docs[1].page_content))

| Varianta | Cum dovedesc la înscriere | Este necesar să susțin testul de limbă POLITEHNICA Bucureşti? |
|:--|:--|:--|
| Liceu absolvit în respectiva limbă de predare | Dacă acest lucru nu apare evident din celelalte documente furnizate la înscriere, am grijă să încarc o copie a unui document eliberat de liceu şi din care să reiasă că am urmat un program de studii în respectiva limbă | NU |
| Certificat de competențe lingvistice pentru respectiva limbă de predare (minim B2); | Încarc o copie a certificatului | NU |
| Disciplină de tip limbă străină promovată în liceu cu media cel puțin 8,00 (pentru clasele a IX- a,a X-a, a XI-a). | Încarc o copie a unui document eliberat de liceul urmat din care reiese acest lucru | NU |
| Proba A, B sau C de la examenul de Bacalaureat; | Încare o copie completă a diplomei de Bacalaureat (inclusiv situaţia la examen) | NU |
| Aleg să susțin testul de limbă POLITEHNICA Bucureşti; | Nu încarc nimic la înscriere | DA |
| L La înscriere optez numai pentru domenii pentru care şcolarizarea se desfăşoară în româna | Nu încarc nimic la înscriere | NU |

### RAG pipeline

Initialize llm

In [35]:
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv(dotenv_path="../.env", override=True)

llm = ChatOpenAI(model=settings.OPENAI_MODEL, temperature=0, top_p=1)

In [36]:
chatbot = Chatbot(retriever=retriever, llm=llm)

In [37]:
chatbot.run("Ce documente sunt necesare pentru pentru inscrierea la un program de studii in limba engleza?")
print(chatbot.refined_question)
# print(chatbot.formatted_prompt)
chatbot._print_response()

Ce documente sunt necesare pentru pentru inscrierea la un program de studii in limba engleza?
UTILIZATOR:
Ce documente sunt necesare pentru pentru inscrierea la un program de studii in limba engleza?
----------------------------------------------------------------------------------------------------------------------------------------------------------------
ASISTENT:
Pentru înscrierea la un program de studii în limba engleză, este necesar să dovediți cunoștințele lingvistice prin una dintre următoarele variante:

1. **Liceu
absolvit în respectiva limbă de predare**: Dacă acest lucru nu este evident din celelalte documente furnizate la înscriere, trebuie să încărcați o copie a unui
document eliberat de liceu care să ateste că ați urmat un program de studii în limba engleză.
   
2. **Certificat de competențe lingvistice pentru respectiva
limbă de predare (minim B2)**: Trebuie să încărcați o copie a certificatului care atestă competențele lingvistice.

3. **Disciplină de tip limbă străin

In [38]:
chatbot.run("Cand are loc testul online de limba de la politehnica?")
print(f"Refined question: {chatbot.refined_question}")
# print(chatbot.formatted_prompt)
chatbot._print_response()

Refined question: Când se va desfășura testul online de limbă străină organizat de Universitatea Politehnica din București?
UTILIZATOR:
Cand are loc testul online de limba de la politehnica?
----------------------------------------------------------------------------------------------------------------------------------------------------------------
ASISTENT:
Testul online de limbă străină organizat de Universitatea Politehnica din București se va desfășura pe data de 21 martie 2025. 

Surse:
- Sursa:
https://admitere.pub.ro/docs/Test%20limba%20straina%2021%2003%20detalii%20S1%202025.pdf, pagina: 1
----------------------------------------------------------------------------------------------------------------------------------------------------------------


In [39]:
# chatbot.run("Cum ma inscriu la testul online de limba de la politehnica?")
# chatbot._print_response()

In [40]:
chatbot.run("Cum ma inscriu la test?")
print(f"Refined question: {chatbot.refined_question}")
chatbot._print_response()

Refined question: Cum pot să mă înscriu la testul online de limbă străină organizat de Universitatea Politehnica din București?
UTILIZATOR:
Cum ma inscriu la test?
----------------------------------------------------------------------------------------------------------------------------------------------------------------
ASISTENT:
Pentru a vă înscrie la testul online de limbă străină organizat de Universitatea Politehnica din București, trebuie să urmați următorii pași:

1. **Înscriere**:
Completați formularul de înscriere disponibil la adresa https://forms.office.com/e/xrMTzESmi7. Este important ca adresa de e-mail pe care o completați în
formular să fie aceeași cu cea utilizată în procesul de înscriere la concurs, pe pagina http://admitere.pub.ro/. 

2. **Termen limită**: Asigurați-vă că
formularul este completat până cel târziu pe 19 martie 2025, ora 12:00. Toate câmpurile din formular sunt obligatorii, iar candidații care nu respectă acest
termen nu vor putea susține testul onlin